In [ ]:
from pynq import (allocate, Overlay)
import numpy, random

ol = Overlay('design_2.bit')
ol.dowload()

In [ ]:
DIM = 128

dma0 = ol.axi_dma_0
dma1 = ol.axi_dma_1
mmult_ip = ol.mmul_0

CTRL_REG = 0x00
AP_START = (1<<0)
AUTO_RESTART = (1<<7)
mmult_ip.register_map.k = DIM
mmult_ip.register_map.m = DIM
mmult_ip.register_map.n = DIM

in_buffer0 = allocate(shape=(DIM, DIM), dtype=np.float32, cacheable=False)
in_buffer1 = allocate(shape=(DIM, DIM), dtype=np.float32, cacheable=False)
out_buffer = allocate(shape=(DIM, DIM), dtype=np.float32, cacheable=False)

In [ ]:
samples0 = np.random.rand(DIM, DIM).astype(dtype=np.float32)
samples1 = np.random.rand(DIM, DIM).astype(dtype=np.float32)
zeros = numpy.zeros((DIM,DIM), dtype=np.float32)

numpy.copyto(in_buffer0, samples0)
numpy.copyto(in_buffer1, samples1)
numpy.copyto(out_buffer, zeros)

In [ ]:
def run_kernel():
    dma0.sendchannel.transfer(in_buffer0)
    dma1.sendchannel.transfer(in_buffer1)
    dma0.recvchannel.transfer(out_buffer)
    mmult_ip.write(CTRL_REG, (AP_START | AUTO_RESTART))
    dma0.sendchannel.wait()
    dma1.sendchannel.wait()
    dma0.recvchannel.wait()

In [ ]:
%%timeit
run_kernel()

In [ ]:
%timeit samples0 @ samples1

In [ ]:
np.array_equal(samples0 @ samples1, out_buffer)